In [1]:
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
data_path = "C://Users//1//Desktop//dataset1" # 数据集所在位置
os.chdir(data_path)
import datetime as dt
from tqdm import tqdm
from func.ts_backtest_framework1 import call_framework_func
from func.utils import ATR_signal_cal,performance_cal

FileNotFoundError: [Errno 2] No such file or directory: 'C://Users//1//Desktop//dataset1'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# 交易规则函数
def my_position_func(symbol, md_data, current_pos, user_data):
    if 'bar_index' not in user_data:
        user_data['bar_index'] = 0
    else:
        user_data['bar_index'] += 1
    target_pos = current_pos

    # 空仓时
    if current_pos == 0:
        user_data['volume'] = 0
        # 当出现交易信号时，按对应信号方向开仓
        if md_data['signal'] != 0:
            target_pos = md_data['signal']
            user_data['open_bar'] = user_data['bar_index']
    # 持有仓位时
    else:
        # 当信号与当前仓位方向相反时，平仓
        if current_pos * md_data['signal'] < 0:
            target_pos = 0
            user_data['open_bar'] = user_data['bar_index']
    return target_pos

In [4]:
def pnl_cal(symbol, values, train_df, N, market_value, my_position_func, rf):
    k1,k2 = values # 确定参数
    # 计算信号
    signal_df = ATR_signal_cal(train_df,N,k1,k2)
    # 回测部分
    daily_df_dict = call_framework_func(symbol,train_df,signal_df,market_value,my_position_func)
    # 交易表现
    port_df, result_df = performance_cal(daily_df_dict[symbol],market_value,rf)

    result = result_df.values[0][0:7].tolist() + result_df.values[0][11:].tolist()
    para_df = pd.DataFrame(result)
    para_df.to_csv(f'output//1.9//para_result//{symbol}_{k1}_{k2}.csv',index=True)


In [5]:
# 通过回溯2020/1/1--2024/1/1各指标确定大致k1,k2,N。
if __name__ == "__main__":
    if not os.path.exists('output//1.9//para_result'): os.makedirs('output//1.9//para_result') # 创建输出文件夹

    # 策略参数优化(以sp500作为研究对象)
    # 回测参数初始化
    symbol = 'sp500'
    #open_commission,close_commission = 0.0001,0.0001 # 定义开平仓手续费
    N, k1, k2, market_value, rf = 14, 0.8, 0.8, 5000000, 0.02     # ATR的rolling天数N,上轨参数k1,下轨参数k2,回测本金和无风险收益率
    ohlc_df = pd.read_csv(f"data//{symbol}_ohlcvm_1min_20190101_20240331.csv", index_col='date_time', parse_dates=['date_time'])

    # 定义k1,k2的参数范围（参数空间）
    k1_values = [i/10 for i in range(1,31)]
    k2_values = [i/10 for i in range(30,0,-1)]
    para_space = [[k1, k2] for k1 in k1_values for k2 in k2_values]

    # 将2020年1月1日至2024年1月1日的数据作为样本内的训练集，2024年1月1日后的数据作为样本外的测试集（样本长度可按需调整）
    train_df = ohlc_df[(ohlc_df.index >= dt.datetime(2024,1,1)) & (ohlc_df.index <= dt.datetime(2024,3,15))].copy()
    test_df = ohlc_df[ohlc_df.index > dt.datetime(2024,3,15)].copy()

    for i,values in enumerate(para_space):
        pnl_cal(symbol, values, train_df, N, market_value, my_position_func, rf)


    # 读取回测结果并画热力图
    # 定义评测指标数据框
    a_r_df = pd.DataFrame(index = k2_values,columns = k1_values)    # 年化收益率
    a_v_df = pd.DataFrame(index = k2_values,columns = k1_values)    # 年化波动率
    s_r_df = pd.DataFrame(index = k2_values,columns = k1_values)    # 夏普比率
    max_dd_df = pd.DataFrame(index = k2_values,columns = k1_values) # 最大回撤
    w_p_df = pd.DataFrame(index = k2_values,columns = k1_values)    # 胜率


    file_list = os.listdir('output//1.9//para_result')
    for file in tqdm(file_list):
        matches = re.findall(r'\d+\.\d+', file)
        k1, k2 = float(matches[0]), float(matches[1])
        # break
        data = pd.read_csv(f'output//1.9//para_result//{file}',index_col=0)
        a_r_df.loc[k2,k1] = data.loc[0,'0']
        a_v_df.loc[k2,k1] = data.loc[1,'0']
        s_r_df.loc[k2,k1] = data.loc[2,'0']
        max_dd_df.loc[k2,k1] = data.loc[3,'0']
        w_p_df.loc[k2,k1] =data.loc[5,'0']


    # plt.rcParams['font.family'] = 'Microsoft YaHei'  # 设置字体名称
    # plt.rcParams['font.size'] = 10  # 可以自定义字体大小
    plt.rcParams['axes.unicode_minus'] = False  # 正确显示负号

    # 创建热力图
    a_r_df = a_r_df.apply(pd.to_numeric, errors='coerce')
    ax = sns.heatmap(a_r_df, cmap="RdYlBu", annot=False, xticklabels=a_r_df.columns,
                      yticklabels=a_r_df.columns[::-1],cbar=True)
    ax.set_xlabel("k1")
    ax.set_ylabel("k2")
    plt.title("年化收益率")
    plt.savefig(f'output//1.9//{symbol}_年化收益率分布.png', dpi=300)
    plt.show()
    plt.close()

    s_r_df = s_r_df.apply(pd.to_numeric, errors='coerce')
    ax1 = sns.heatmap(s_r_df, cmap="RdYlBu", annot=False, xticklabels=s_r_df.column, yticklabels=s_r_df.columns[::-1], cbar=True)
    ax1.set_xlabel("k1")
    ax1.set_ylabel("k2")
    plt.title("夏普比率")
    plt.savefig(f'output//1.9//{symbol}_夏普比率分布.png', dpi=300)
    plt.show()
    plt.close()

    max_dd_df = max_dd_df.apply(pd.to_numeric, errors='coerce')
    ax2 = sns.heatmap(max_dd_df, cmap="RdYlBu", annot=False, xticklabels=max_dd_df.columns, yticklabels=max_dd_df.columns[::-1], cbar=True)
    ax2.set_xlabel("k1")
    ax2.set_ylabel("k2")
    plt.title("最大回撤")
    plt.savefig(f'output//1.9//{symbol}_最大回撤分布.png', dpi=300)
    plt.show()
    plt.close()

    w_p_df = w_p_df.apply(pd.to_numeric, errors='coerce')
    ax5 = sns.heatmap(w_p_df, cmap="RdYlBu", annot=False, xticklabels=w_p_df.columns, yticklabels=w_p_df.columns[::-1], cbar=True)
    ax5.set_xlabel("k1")
    ax5.set_ylabel("k2")
    plt.title("胜率")
    plt.savefig(f'output//1.9//{symbol}_胜率分布.png', dpi=300)
    plt.show()
    plt.close()


    # 在该段时间内将k1 =2.9,k2=0.4作为样本内最优参数进行样本外测试
    k1,k2 = 2.9,0.4
    signal_df = ATR_signal_cal(test_df,N,k1,k2)
    daily_df_dict = call_framework_func(symbol, test_df, signal_df, market_value, my_position_func)

    # 交易表现
    port_df, result_df = performance_cal(daily_df_dict[symbol],market_value,rf)
    print(result_df)
    # 画图
    fig, ax1 = plt.subplots()

    # 净值曲线
    ax1.plot(port_df['nv'], label='Net Value', color='b')
    ax1.set_ylabel('Net Value', color='b')
    ax1.tick_params(axis='y', labelcolor='b')
    ax1.legend(loc='upper left', frameon=False)

    # 最大回撤
    ax2 = ax1.twinx()
    ax2.plot(port_df['max_dd'], label='Max Drawdown', color='g')
    ax2.set_ylabel('Max Drawdown', color='g')
    ax2.tick_params(axis='y', labelcolor='g')
    ax2.legend(loc='upper right', frameon=False)

    # 显示图表
    plt.title(f'{symbol} backtest result')
    plt.tight_layout()
    plt.savefig(f'output//1.9//{symbol}_ATR_调参后回测收益.png', dpi=300)
    plt.show()
    plt.close()

FileNotFoundError: [Errno 2] No such file or directory: 'data//sp500_ohlcvm_1min_20190101_20240331.csv'